In [1]:
import digitalhub as dh

project = dh.get_or_create_project("project-ml-ci")

In [2]:
import os
new_folder = 'src'
if not os.path.exists(new_folder):
    os.makedirs(new_folder)

In [3]:
%%writefile "src/data-prep.py"

import pandas as pd
from sklearn.datasets import load_breast_cancer

from digitalhub_runtime_python import handler

@handler(outputs=["dataset"])
def breast_cancer_generator():
    """
    A function which generates the breast cancer dataset
    """
    breast_cancer = load_breast_cancer()
    breast_cancer_dataset = pd.DataFrame(
        data=breast_cancer.data, columns=breast_cancer.feature_names
    )
    breast_cancer_labels = pd.DataFrame(data=breast_cancer.target, columns=["target"])
    breast_cancer_dataset = pd.concat(
        [breast_cancer_dataset, breast_cancer_labels], axis=1
    )

    return breast_cancer_dataset

Writing src/data-prep.py


In [4]:
data_gen_fn = project.new_function(name="data-prep",
                                   kind="python",
                                   python_version="PYTHON3_10",
                                   code_src="src/data-prep.py",
                                   handler="breast_cancer_generator")

In [5]:
gen_data_run = data_gen_fn.run(action="job", outputs={"dataset": "dataset"}, local_execution=False)

In [6]:
gen_data_run.refresh()

{'kind': 'python+run', 'metadata': {'project': 'project-ml-ci', 'name': '2de5a902-0d00-41ec-bf7e-86729e3ed8ba', 'created': '2024-10-25T07:54:39.193Z', 'updated': '2024-10-25T07:54:44.231Z', 'created_by': 'khurshid@fbk.eu', 'updated_by': 'khurshid@fbk.eu'}, 'spec': {'task': 'python+job://project-ml-ci/data-prep:198e8387-0b03-4828-8eb7-914e10e4771e', 'local_execution': False, 'function': 'python://project-ml-ci/data-prep:198e8387-0b03-4828-8eb7-914e10e4771e', 'source': {'source': 'src/data-prep.py', 'handler': 'breast_cancer_generator', 'base64': 'CmltcG9ydCBwYW5kYXMgYXMgcGQKZnJvbSBza2xlYXJuLmRhdGFzZXRzIGltcG9ydCBsb2FkX2JyZWFzdF9jYW5jZXIKCmZyb20gZGlnaXRhbGh1Yl9ydW50aW1lX3B5dGhvbiBpbXBvcnQgaGFuZGxlcgoKQGhhbmRsZXIob3V0cHV0cz1bImRhdGFzZXQiXSkKZGVmIGJyZWFzdF9jYW5jZXJfZ2VuZXJhdG9yKCk6CiAgICAiIiIKICAgIEEgZnVuY3Rpb24gd2hpY2ggZ2VuZXJhdGVzIHRoZSBicmVhc3QgY2FuY2VyIGRhdGFzZXQKICAgICIiIgogICAgYnJlYXN0X2NhbmNlciA9IGxvYWRfYnJlYXN0X2NhbmNlcigpCiAgICBicmVhc3RfY2FuY2VyX2RhdGFzZXQgPSBwZC5EYXRhRnJhbWUoCiAg

In [7]:
gen_data_run.output("dataset").as_df().head()

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension,target
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,0
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,0
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,0
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,0
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,0


In [8]:
%%writefile "src/train-model.py"


import pandas as pd
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split

from digitalhub_runtime_python import handler
from sklearn.svm import SVC
from pickle import dump
import sklearn.metrics
import os

@handler(outputs=["dataset"])
def train(project, di):

    df_cancer = di.as_df()
    X = df_cancer.drop(['target'],axis=1)
    y = df_cancer['target']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state=5)
    svc_model = SVC()
    svc_model.fit(X_train, y_train)
    y_predict = svc_model.predict(X_test)

    if not os.path.exists("model"):
        os.makedirs("model")

    with open("model/cancer_classifier.pkl", "wb") as f:
        dump(svc_model, f, protocol=5)

    metrics = {
        "f1_score": sklearn.metrics.f1_score(y_test, y_predict),
        "accuracy": sklearn.metrics.accuracy_score(y_test, y_predict),
        "precision": sklearn.metrics.precision_score(y_test, y_predict),
        "recall": sklearn.metrics.recall_score(y_test, y_predict),
    }
    project.log_model(
            name="cancer_classifier",
            kind="sklearn",
            source="./model/",
            metrics=metrics
    )

Writing src/train-model.py


In [9]:
train_fn = project.new_function(name="train",
                                kind="python",
                                python_version="PYTHON3_10",
                                code_src="src/train-model.py",
                                handler="train",
                                requirements=["scikit-learn==1.2.2"])

In [10]:
train_run = train_fn.run(action="job", inputs={"di": gen_data_run.output("dataset").key})

In [11]:
model = project.get_model("cancer_classifier")
model.spec.path

's3://datalake/project-ml-ci/model/cancer_classifier/995a28fc-283c-4ab5-8351-c2425b9e2e0e/'

In [12]:
func = project.new_function(name="serve_sklearnmodel",
                            kind="sklearnserve",
                            path=model.spec.path + 'model/cancer_classifier.pkl')

serve_run = func.run(action="serve")

Let us wait till the 'serve' is finished 

In [17]:
serve_run.refresh().status

{'state': 'RUNNING', 'message': 'deployment d-sklearnserveserve-e3f62ae5-41cd-465b-9784-531907a3aaec created, service s-sklearnserveserve-e3f62ae5-41cd-465b-9784-531907a3aaec created', 'transitions': [{'status': 'RUNNING', 'message': 'deployment d-sklearnserveserve-e3f62ae5-41cd-465b-9784-531907a3aaec created, service s-sklearnserveserve-e3f62ae5-41cd-465b-9784-531907a3aaec created', 'time': '2024-10-25T07:56:35.897665387Z'}, {'status': 'READY', 'time': '2024-10-25T07:56:35.758802367Z'}, {'status': 'BUILT', 'time': '2024-10-25T07:56:35.745445087Z'}], 'k8s': {'pods': [{'metadata': {'creationTimestamp': 1729842995.0, 'generateName': 'd-sklearnserveserve-e3f62ae5-41cd-465b-9784-531907a3aaec-5f975c88dd-', 'labels': {'app.kubernetes.io/instance': 'dhcore-e3f62ae5-41cd-465b-9784-531907a3aaec', 'app.kubernetes.io/managed-by': 'dhcore', 'app.kubernetes.io/part-of': 'dhcore-project-ml-ci', 'app.kubernetes.io/version': 'e3f62ae5-41cd-465b-9784-531907a3aaec', 'dhcore/framework': 'k8sserve', 'dhco

In [18]:
import numpy as np

data = np.random.rand(2, 30).tolist()
json = {
    "inputs": [
        {
        "name": "input-0",
        "shape": [2, 30],
        "datatype": "FP32",
        "data": data
        }
    ]
}

In [19]:
serve_run.invoke(json=json).json()

{'model_name': 'model',
 'id': 'eb1e4d3e-56f1-4693-9cd6-ea0c13b20c5e',
 'parameters': {},
 'outputs': [{'name': 'predict',
   'shape': [2, 1],
   'datatype': 'INT64',
   'parameters': {'content_type': 'np'},
   'data': [1, 1]}]}

In [36]:
%%writefile "src/dbt_ml_pipeline.py"

from digitalhub_runtime_kfp.dsl import pipeline_context

def myhandler(): #path
    with pipeline_context() as pc:
        downloader = pc.step(name="download-data", function="data-prep", action="job", outputs={"dataset": "dataset"})
        trainer = pc.step(name="train-model", function="train", action="job", inputs={"di": downloader.outputs["dataset"]})
        # serve = pc.step(name="serve", function="serve_sklearnmodel", action="serve", inputs={"path":path})

Overwriting src/dbt_ml_pipeline.py


In [37]:
workflow = project.new_workflow(name="pipeline_ml", kind="kfp", code_src= "src/dbt_ml_pipeline.py", handler = "myhandler")

In [39]:
# workflow.run(parameters={"path": model.spec.path + 'model/cancer_classifier.pkl'})
workflow.run()

{'kind': 'kfp+run', 'metadata': {'project': 'project-ml-ci', 'name': '5242c0a0-a22a-414c-9ac3-c132e8c70cc9', 'created': '2024-10-25T08:25:37.619Z', 'updated': '2024-10-25T08:25:37.801Z', 'created_by': 'khurshid@fbk.eu', 'updated_by': 'khurshid@fbk.eu'}, 'spec': {'task': 'kfp+pipeline://project-ml-ci/pipeline_ml:34e65c92-18cb-4f03-931c-6055fc908487', 'local_execution': False, 'function': 'kfp://project-ml-ci/pipeline_ml:34e65c92-18cb-4f03-931c-6055fc908487', 'source': {'source': 'src/dbt_ml_pipeline.py', 'handler': 'myhandler', 'base64': 'CmZyb20gZGlnaXRhbGh1Yl9ydW50aW1lX2tmcC5kc2wgaW1wb3J0IHBpcGVsaW5lX2NvbnRleHQKCmRlZiBteWhhbmRsZXIocGF0aCk6CiAgICB3aXRoIHBpcGVsaW5lX2NvbnRleHQoKSBhcyBwYzoKICAgICAgICBkb3dubG9hZGVyID0gcGMuc3RlcChuYW1lPSJkb3dubG9hZC1kYXRhIiwgZnVuY3Rpb249ImRhdGEtcHJlcCIsIGFjdGlvbj0iam9iIiwgb3V0cHV0cz17ImRhdGFzZXQiOiAiZGF0YXNldCJ9KQogICAgICAgIHRyYWluZXIgPSBwYy5zdGVwKG5hbWU9InRyYWluLW1vZGVsIiwgZnVuY3Rpb249InRyYWluIiwgYWN0aW9uPSJqb2IiLCBpbnB1dHM9eyJkaSI6IGRvd25sb2FkZXIub3V0cHV0